In [14]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.linear_model import LogisticRegression

path = Path('data', 'weatherAUS_clean.csv')
df = pd.read_csv(path)

In [15]:
# split into train and test
label = df.pop('RainTomorrow_Yes')
train_X, test_X, train_y, test_y = train_test_split(df, label, test_size= 0.2)

In [16]:
clf = LogisticRegression(random_state=0).fit(train_X, train_y)
clf.predict(test_X)


array([0, 1, 0, ..., 0, 0, 0], dtype=int64)